In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

import numpy as np

In [2]:
# Loading dataset
df = pd.read_excel("https://getaround-bucket-27-12-2022.s3.eu-west-3.amazonaws.com/get_around_delay_analysis.xlsx")
df.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


## Basic exploration of the dataset

In [3]:
# Checking data types
print(df.dtypes)

# Getting an overview of the dataset
df.describe(include="all")

rental_id                                       int64
car_id                                          int64
checkin_type                                   object
state                                          object
delay_at_checkout_in_minutes                  float64
previous_ended_rental_id                      float64
time_delta_with_previous_rental_in_minutes    float64
dtype: object


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


In [4]:
# Checking the percentage of missing values
100*df.isnull().sum()/df.shape[0]

rental_id                                      0.000000
car_id                                         0.000000
checkin_type                                   0.000000
state                                          0.000000
delay_at_checkout_in_minutes                  23.294228
previous_ended_rental_id                      91.360863
time_delta_with_previous_rental_in_minutes    91.360863
dtype: float64

In [5]:
# Getting the number of cars in the dataset
print("Number of cars: ", df["car_id"].nunique())

# Getting the number of rentals in the dataset
print("Number of rentals: ", df["rental_id"].nunique())

Number of cars:  8143
Number of rentals:  21310


## Distribution per check-in type

In [6]:
# Creating a donut chart to visualize percentages of connect and mobile check-in types
colors = ['#8d1586', '#eec186']

# Use Plotly Express to create a donut chart
fig = px.pie(df, names='checkin_type', color_discrete_sequence=colors, hole=0.55)

# Add a title to the chart
fig.update_layout(title='Car rentals by check-in type')

# Show the chart
fig.show()

## Cancellations and delays by contract type

In [7]:
# Computing the percentage of rentals that ended or were canceled for each contract type
contract_ended = df[df['state'] == 'ended']
contract_canceled = df[df['state'] == 'canceled']
mobile_ended_pct = round(100 * len(contract_ended[contract_ended['checkin_type'] == 'mobile']) / len(df[df['checkin_type'] == 'mobile']), 2)
mobile_canceled_pct = round(100 * len(contract_canceled[contract_canceled['checkin_type'] == 'mobile']) / len(df[df['checkin_type'] == 'mobile']), 2)
connect_ended_pct = round(100 * len(contract_ended[contract_ended['checkin_type'] == 'connect']) / len(df[df['checkin_type'] == 'connect']), 2)
connect_canceled_pct = round(100 * len(contract_canceled[contract_canceled['checkin_type'] == 'connect']) / len(df[df['checkin_type'] == 'connect']), 2)
total_ended_pct = round(100 * len(contract_ended) / len(df), 2)
total_canceled_pct = round(100 * len(contract_canceled) / len(df), 2)

# Creating a new dataframe with the percentages computed above
df_pct = pd.DataFrame({
    'Contract Type': ['Mobile', 'Connect', 'Overall'],
    'Ended': [mobile_ended_pct, connect_ended_pct, total_ended_pct],
    'Canceled': [mobile_canceled_pct, connect_canceled_pct, total_canceled_pct]
})

# Creating a horizontal stacked bar chart
fig = go.Figure()
fig.add_trace(go.Bar(y=df_pct['Contract Type'], x=df_pct['Canceled'], orientation='h', name='Canceled', marker=dict(color='#c71414'), text=[f"{mobile_canceled_pct}%", f"{connect_canceled_pct}%", f"{total_canceled_pct}%"],
                     textposition='auto', textfont=dict(color='white')))
fig.add_trace(go.Bar(y=df_pct['Contract Type'], x=df_pct['Ended'], orientation='h', name='Ended', marker=dict(color='#7b728e'), text=[f"{mobile_ended_pct}%", f"{connect_ended_pct}%", f"{total_ended_pct}%"],
                     textposition='auto', textfont=dict(color='white')))
fig.update_layout(barmode='stack', title='How many contracts were canceled?', yaxis={'categoryorder':'total ascending'})

fig.show()

In [8]:
df["delay_at_checkout"] = df["delay_at_checkout_in_minutes"].apply(lambda x: "checkout delayed" if x>0 else "checkout not delayed")

In [9]:
# Computing the percentage of rentals were delayed for each contract type
checkout_delayed = df[df['delay_at_checkout'] == 'checkout delayed']
checkout_not_delayed = df[df['delay_at_checkout'] == 'checkout not delayed']

mobile_delayed_pct = round(100 * len(checkout_delayed[checkout_delayed['checkin_type'] == 'mobile']) / len(df[df['checkin_type'] == 'mobile']), 2)
mobile_not_delayed_pct = round(100 * len(checkout_not_delayed[checkout_not_delayed['checkin_type'] == 'mobile']) / len(df[df['checkin_type'] == 'mobile']), 2)
connect_delayed_pct = round(100 * len(checkout_delayed[checkout_delayed['checkin_type'] == 'connect']) / len(df[df['checkin_type'] == 'connect']), 2)
connect_not_delayed_pct = round(100 * len(checkout_not_delayed[checkout_not_delayed['checkin_type'] == 'connect']) / len(df[df['checkin_type'] == 'connect']), 2)
total_delayed_pct = round(100 * len(checkout_delayed) / len(df), 2)
total_not_delayed_pct = round(100 * len(checkout_not_delayed) / len(df), 2)

# Creating a new dataframe with the percentages computed above
df_pct = pd.DataFrame({
    'Contract Type': ['Mobile', 'Connect', 'Overall'],
    'Delayed': [mobile_delayed_pct, connect_delayed_pct, total_delayed_pct],
    'Not Delayed': [mobile_not_delayed_pct, connect_not_delayed_pct, total_not_delayed_pct]
})

# Creating a horizontal stacked bar chart
fig = go.Figure()
fig.add_trace(go.Bar(y=df_pct['Contract Type'], x=df_pct['Delayed'], orientation='h', name='Delayed', marker=dict(color='#c71414'), text=[f"{mobile_delayed_pct}%", f"{connect_delayed_pct}%", f"{total_delayed_pct}%"],
                     textposition='auto', textfont=dict(color='white')))
fig.add_trace(go.Bar(y=df_pct['Contract Type'], x=df_pct['Not Delayed'], orientation='h', name='Not Delayed', marker=dict(color='#7b728e'), text=[f"{mobile_not_delayed_pct}%", f"{connect_not_delayed_pct}%", f"{total_not_delayed_pct}%"],
                     textposition='auto', textfont=dict(color='white')))
fig.update_layout(barmode='stack', title='How many clients were late to return the car?', yaxis={'categoryorder':'total ascending'})

fig.show()

### Deltas between rentals

In [10]:
df_grouped = df.groupby(by="time_delta_with_previous_rental_in_minutes").size().reset_index()
df_grouped.rename(columns={0:'count'}, inplace=True)
df_grouped["percentage"] = df_grouped["count"]/df_grouped["count"].sum() * 100
df_grouped["cumulated_percentage"] = 0
df_grouped.loc[0, "cumulated_percentage"] = df_grouped.loc[0, "percentage"]
for i in range(1, len(df_grouped)):
    df_grouped.loc[i, "cumulated_percentage"] = df_grouped.loc[i, "percentage"] + df_grouped.loc[i-1, "cumulated_percentage"]
df_grouped.head()

,time_delta_with_previous_rental_in_minutes,count,percentage,cumulated_percentage
0,0.0,279,15.154807,15.154807
1,30.0,122,6.626833,21.781640
2,60.0,183,9.940250,31.721890
3,90.0,82,4.454101,36.175991
4,120.0,137,7.441608,43.617599


In [11]:
for i in range(1, len(df_grouped)):
    df_grouped.loc[i, "cumulated_percentage"] = df_grouped.loc[i, "percentage"] + df_grouped.loc[i-1, "cumulated_percentage"]
fig = px.bar(df_grouped, x="time_delta_with_previous_rental_in_minutes", y="count", title="Currently Existing Deltas")
fig.update_traces(marker_color="#5ecbdd")
fig.show()

In [12]:
fig = px.bar(df_grouped, x='time_delta_with_previous_rental_in_minutes', y='cumulated_percentage')
fig.update_traces(marker_color="#5ecbdd")
fig.update_layout(title='What percentage of contracts will be impacted by a given minimum delta?', xaxis=dict(title='Delta'), yaxis=dict(title='Percent'), bargap=0.10)
fig.show()

### Delays at checkout

In [13]:
# Using violin plot to get a first look at the distribution
fig = px.violin(df, y="delay_at_checkout_in_minutes")
fig.show()

It seems that most delays at checkout are between -4000 minutes and 7000 minutes, but there is a certain number of outliers.

In [14]:
delays_df = df[df['delay_at_checkout_in_minutes']>0]
delays_df.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout
2,508131,359049,connect,ended,70.0,NaN,NaN,checkout delayed
9,513434,256528,connect,ended,23.0,NaN,NaN,checkout delayed
12,514257,402596,mobile,ended,214.0,NaN,NaN,checkout delayed
13,515147,257466,mobile,ended,15.0,NaN,NaN,checkout delayed
17,518895,398816,mobile,ended,66.0,NaN,NaN,checkout delayed


In [15]:
len(delays_df)

9404

In [16]:
delays_df.nlargest(15, "delay_at_checkout_in_minutes", keep='first')

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout
1021,532240,379042,mobile,ended,71084.0,NaN,NaN,checkout delayed
1787,549980,402829,mobile,ended,64059.0,NaN,NaN,checkout delayed
13776,537546,408750,mobile,ended,37382.0,NaN,NaN,checkout delayed
1038,534699,270195,mobile,ended,23126.0,NaN,NaN,checkout delayed
5257,556557,413591,mobile,ended,22618.0,NaN,NaN,checkout delayed
153,540733,393558,mobile,ended,15422.0,NaN,NaN,checkout delayed
9080,554989,366198,mobile,ended,14408.0,NaN,NaN,checkout delayed
15485,552196,278701,mobile,ended,13463.0,NaN,NaN,checkout delayed
1427,546459,297511,mobile,ended,12968.0,NaN,NaN,checkout delayed
13446,539642,396000,mobile,ended,10323.0,NaN,NaN,checkout delayed


In [17]:
# Creating a histogram to show distribution of delays
trace = go.Histogram(x=delays_df['delay_at_checkout_in_minutes'], nbinsx=90, marker=dict(color='blue'))

layout = go.Layout(title='How are drivers delays distributed?', xaxis=dict(title='Delay at Checkout in Minutes'), yaxis=dict(title='Count'))

fig = go.Figure(data=[trace], layout=layout)

fig.update_layout(bargap=0.15)

fig.show()

Most delays were shorter than 1000 minutes. Let us have a closer look at this group:

In [18]:
less_than_1000_min_delays = delays_df[delays_df['delay_at_checkout_in_minutes']<=1000]

trace = go.Histogram(x=less_than_1000_min_delays['delay_at_checkout_in_minutes'], nbinsx=100, marker=dict(color='blue'))

layout = go.Layout(title='How are drivers delays distributed?', xaxis=dict(title='Delay at Checkout in Minutes'), yaxis=dict(title='Count'))

fig = go.Figure(data=[trace], layout=layout)
fig.update_traces(marker_color="#d076ca")
fig.update_layout(bargap=0.15)

fig.show()

To help visualize what minimum deltas between rentals could be proposed, we will look at what percentage of delays are shorter than a certain  threshold. For instance, if 80 % of delays are shorter than 180 minutes, then an imposed minimum delta of 180 minutes between rentals will supposedly compensate for 80 % of delays.

In [19]:
thresholds = range(0, 1000, 10)

# Creating bins for the time duration column
bins = pd.cut(df['delay_at_checkout_in_minutes'], bins=thresholds, include_lowest=True)

# Calculate the percentage of events falling in each bin
bin_counts = bins.value_counts(normalize=True, sort=False) * 100

# Print the results
print(bin_counts)

(-0.001, 10.0]    16.817143
(10.0, 20.0]      11.595779
(20.0, 30.0]       9.093876
(30.0, 40.0]       7.451322
(40.0, 50.0]       5.906668
                    ...    
(940.0, 950.0]     0.065267
(950.0, 960.0]     0.032634
(960.0, 970.0]     0.054389
(970.0, 980.0]     0.021756
(980.0, 990.0]     0.010878
Name: delay_at_checkout_in_minutes, Length: 99, dtype: float64


In [20]:
delay_percentages = bin_counts.to_frame().reset_index()
delay_percentages.rename(columns={"delay_at_checkout_in_minutes":"percentage", "index":"interval"}, inplace=True)
delay_percentages.head()

,interval,percentage
0,"(-0.001, 10.0]",16.817143
1,"(10.0, 20.0]",11.595779
2,"(20.0, 30.0]",9.093876
3,"(30.0, 40.0]",7.451322
4,"(40.0, 50.0]",5.906668


In [21]:
delay_percentages["cumulated_percentage"]=0
delay_percentages["delta_threshold"]=10
delay_percentages.loc[0, "cumulated_percentage"] = delay_percentages.loc[0, "percentage"]
display(delay_percentages.head())
for i in range(1, len(delay_percentages)):
    delay_percentages.loc[i, "cumulated_percentage"] = delay_percentages.loc[i, "percentage"] + delay_percentages.loc[i-1, "cumulated_percentage"]
    delay_percentages.loc[i, "delta_threshold"] = delay_percentages.loc[i-1, "delta_threshold"] + 10

,interval,percentage,cumulated_percentage,delta_threshold
0,"(-0.001, 10.0]",16.817143,16.817143,10
1,"(10.0, 20.0]",11.595779,0.000000,10
2,"(20.0, 30.0]",9.093876,0.000000,10
3,"(30.0, 40.0]",7.451322,0.000000,10
4,"(40.0, 50.0]",5.906668,0.000000,10


In [22]:
display(delay_percentages.head(20))

,interval,percentage,cumulated_percentage,delta_threshold
0,"(-0.001, 10.0]",16.817143,16.817143,10
1,"(10.0, 20.0]",11.595779,28.412923,20
2,"(20.0, 30.0]",9.093876,37.506799,30
3,"(30.0, 40.0]",7.451322,44.958120,40
4,"(40.0, 50.0]",5.906668,50.864788,50
5,"(50.0, 60.0]",5.047319,55.912107,60
6,"(60.0, 70.0]",5.025563,60.937670,70
7,"(70.0, 80.0]",3.872512,64.810182,80
8,"(80.0, 90.0]",3.383009,68.193190,90
9,"(90.0, 100.0]",3.045796,71.238986,100


In [23]:
fig = px.bar(delay_percentages, x='delta_threshold', y='cumulated_percentage')
fig.update_layout(title='What percentage of delays can be offset by a given delta?', xaxis=dict(title='Delta'), yaxis=dict(title='Percent'), bargap=0.10)
fig.update_traces(marker_color="#2ea2ea")
fig.show()


## Impact of delay on the next client

Unfortunately, for most rentals the id of the previous rental is not known which makes it more difficult to establish whether a client's delay at checkout had an impact on the next client.
For further analysis, we will concentrate on the cases where the id of the previous rental is known and so the impact of the next client can be estimated.

In [24]:
# # Saving ids of of previous known rentals as a series
known_previous_rentals = df[df["previous_ended_rental_id"].notnull()]["previous_ended_rental_id"]

known_previous_rentals.head()

6     563782.0
19    545639.0
23    537298.0
34    510607.0
40    557404.0
Name: previous_ended_rental_id, dtype: float64

In [25]:
# Transferring the delay of the previous client to the row of the next client
# in the new column "delay_of_the_previous_client_in_minutes"
for i in range(0, len(df)):
    if df["previous_ended_rental_id"].iloc[i] in known_previous_rentals.values:
        previous_client_id = df["previous_ended_rental_id"].iloc[i]
        delay_of_previous_client = df.loc[df['rental_id'] == df['previous_ended_rental_id'].iloc[i]]["delay_at_checkout_in_minutes"].loc[df.loc[df['rental_id'] == df['previous_ended_rental_id'].iloc[i]].index].values[0]
        #print(delay_of_previous_client)
        df.loc[i, "delay_of_previous_client_in_minutes"] = delay_of_previous_client
    else: 
        pass


df.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout,delay_of_previous_client_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN,checkout not delayed,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN,checkout not delayed,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN,checkout delayed,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN,checkout not delayed,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN,checkout not delayed,NaN


We will create a new column showing whether the previous client was late at the checkout or not.

In [26]:
df["previous_client_late"] = df["delay_of_previous_client_in_minutes"].apply(lambda x: "yes" if x>0 else "no")

As there a time delta between some rentals, it can be supposed that when a previous client's delay at checkout is smaller than the delta before the next client's rental, the delay won't have an impact on the next client, so even if the next client cancels, such cancellation cannot be attributed to the delay.

For the same reason, when a client returns a car in advance and the next client still cancels, such cancellations cannot be attributed to the previous driver's delay, so such cancellations are out of the scope of this analysis.

In [27]:
df["delay_of_rental_start_in_minutes"] = df["delay_of_previous_client_in_minutes"] - df["time_delta_with_previous_rental_in_minutes"]

df.head(12)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout,delay_of_previous_client_in_minutes,previous_client_late,delay_of_rental_start_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN,checkout not delayed,NaN,no,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN,checkout not delayed,NaN,no,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN,checkout delayed,NaN,no,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN,checkout not delayed,NaN,no,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN,checkout not delayed,NaN,no,NaN
5,511626,398802,mobile,ended,-203.0,NaN,NaN,checkout not delayed,NaN,no,NaN
6,511639,370585,connect,ended,-15.0,563782.0,570.0,checkout not delayed,136.0,yes,-434.0
7,512303,371242,mobile,ended,-44.0,NaN,NaN,checkout not delayed,NaN,no,NaN
8,512475,322502,mobile,canceled,NaN,NaN,NaN,checkout not delayed,NaN,no,NaN
9,513434,256528,connect,ended,23.0,NaN,NaN,checkout delayed,NaN,no,NaN


We'll create a new column 'client_rental_start_impacted' to show whether a client was impacted or not by the delay of the previous driver.

In [28]:
df["client_rental_start_impacted"] = df["delay_of_rental_start_in_minutes"].apply(lambda x: "yes" if x>0 else "no")

df.head(15)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout,delay_of_previous_client_in_minutes,previous_client_late,delay_of_rental_start_in_minutes,client_rental_start_impacted
0,505000,363965,mobile,canceled,NaN,NaN,NaN,checkout not delayed,NaN,no,NaN,no
1,507750,269550,mobile,ended,-81.0,NaN,NaN,checkout not delayed,NaN,no,NaN,no
2,508131,359049,connect,ended,70.0,NaN,NaN,checkout delayed,NaN,no,NaN,no
3,508865,299063,connect,canceled,NaN,NaN,NaN,checkout not delayed,NaN,no,NaN,no
4,511440,313932,mobile,ended,NaN,NaN,NaN,checkout not delayed,NaN,no,NaN,no
5,511626,398802,mobile,ended,-203.0,NaN,NaN,checkout not delayed,NaN,no,NaN,no
6,511639,370585,connect,ended,-15.0,563782.0,570.0,checkout not delayed,136.0,yes,-434.0,no
7,512303,371242,mobile,ended,-44.0,NaN,NaN,checkout not delayed,NaN,no,NaN,no
8,512475,322502,mobile,canceled,NaN,NaN,NaN,checkout not delayed,NaN,no,NaN,no
9,513434,256528,connect,ended,23.0,NaN,NaN,checkout delayed,NaN,no,NaN,no


In [29]:
prev_df = df.dropna(subset=['previous_ended_rental_id']).reset_index(drop=True)

prev_df.head(15)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout,delay_of_previous_client_in_minutes,previous_client_late,delay_of_rental_start_in_minutes,client_rental_start_impacted
0,511639,370585,connect,ended,-15.0,563782.0,570.0,checkout not delayed,136.0,yes,-434.0,no
1,519491,312389,mobile,ended,58.0,545639.0,420.0,checkout delayed,140.0,yes,-280.0,no
2,521156,392479,mobile,ended,NaN,537298.0,0.0,checkout not delayed,NaN,no,NaN,no
3,525044,349751,mobile,ended,NaN,510607.0,60.0,checkout not delayed,-113.0,no,-173.0,no
4,528808,181625,connect,ended,-76.0,557404.0,330.0,checkout not delayed,-352.0,no,-682.0,no
5,533670,320824,connect,ended,-6.0,556563.0,630.0,checkout not delayed,23.0,yes,-607.0,no
6,534827,404169,mobile,ended,-7.0,531158.0,90.0,checkout not delayed,29.0,yes,-61.0,no
7,535611,397814,mobile,ended,68.0,535512.0,300.0,checkout delayed,44.0,yes,-256.0,no
8,535770,352436,mobile,ended,74.0,524703.0,60.0,checkout delayed,7.0,yes,-53.0,no
9,537576,397470,mobile,ended,18.0,539005.0,0.0,checkout delayed,-29.0,no,-29.0,no


In [30]:
total_rentals = len(prev_df)

In [31]:
delayed_df=prev_df[prev_df["previous_client_late"]=="yes"]

len(delayed_df)

display(delayed_df.head())

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout,delay_of_previous_client_in_minutes,previous_client_late,delay_of_rental_start_in_minutes,client_rental_start_impacted
0,511639,370585,connect,ended,-15.0,563782.0,570.0,checkout not delayed,136.0,yes,-434.0,no
1,519491,312389,mobile,ended,58.0,545639.0,420.0,checkout delayed,140.0,yes,-280.0,no
5,533670,320824,connect,ended,-6.0,556563.0,630.0,checkout not delayed,23.0,yes,-607.0,no
6,534827,404169,mobile,ended,-7.0,531158.0,90.0,checkout not delayed,29.0,yes,-61.0,no
7,535611,397814,mobile,ended,68.0,535512.0,300.0,checkout delayed,44.0,yes,-256.0,no


In [32]:
# Compute the percentage of rentals were impacted by delay for each contract type
impacted = delayed_df[delayed_df['client_rental_start_impacted'] == 'yes']
not_impacted = delayed_df[delayed_df['client_rental_start_impacted'] == 'no']

mobile_impacted_pct = round(100 * len(impacted[impacted['checkin_type'] == 'mobile']) / len(delayed_df[delayed_df['checkin_type'] == 'mobile']), 2)
mobile_not_impacted_pct = round(100 * len(not_impacted[not_impacted['checkin_type'] == 'mobile']) / len(delayed_df[delayed_df['checkin_type'] == 'mobile']), 2)
connect_impacted_pct = round(100 * len(impacted[impacted['checkin_type'] == 'connect']) / len(delayed_df[delayed_df['checkin_type'] == 'connect']), 2)
connect_not_impacted_pct = round(100 * len(not_impacted[not_impacted['checkin_type'] == 'connect']) / len(delayed_df[delayed_df['checkin_type'] == 'connect']), 2)
total_impacted_pct = round(100 * len(impacted) / len(delayed_df), 2)
total_not_impacted_pct = round(100 * len(not_impacted) / len(delayed_df), 2)

# Create a new dataframe with the percentages computed above
df_pct = pd.DataFrame({
    'Contract Type': ['Mobile', 'Connect', 'Overall'],
    'Impacted': [mobile_impacted_pct, connect_impacted_pct, total_impacted_pct],
    'Not Impacted': [mobile_not_impacted_pct, connect_not_impacted_pct, total_not_impacted_pct]
})

# Use Plotly to create a horizontal stacked bar chart
fig = go.Figure()
fig.add_trace(go.Bar(y=df_pct['Contract Type'], x=df_pct['Impacted'], orientation='h', name='Impacted', marker=dict(color='#c71414'), text=[f"{mobile_impacted_pct}%", f"{connect_impacted_pct}%", f"{total_impacted_pct}%"],
                     textposition='auto', textfont=dict(color='white')))
fig.add_trace(go.Bar(y=df_pct['Contract Type'], x=df_pct['Not Impacted'], orientation='h', name='Not Impacted', marker=dict(color='#7b728e'), text=[f"{mobile_not_impacted_pct}%", f"{connect_not_impacted_pct}%", f"{total_not_impacted_pct}%"],
                     textposition='auto', textfont=dict(color='white')))
fig.update_layout(barmode='stack', title='In cases where the previous driver was late, how many clients were impacted by the delay?', yaxis={'categoryorder':'total ascending'})

# Show the chart
fig.show()

It looks like in roughly three quarters of cases, the existing deltas between rentals are sufficient to offset the delays of the drivers.

In [33]:
impacted_df = prev_df[prev_df['client_rental_start_impacted']=="yes"]
impacted_df

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout,delay_of_previous_client_in_minutes,previous_client_late,delay_of_rental_start_in_minutes,client_rental_start_impacted
11,540479,374684,mobile,ended,12.0,539751.0,0.0,checkout delayed,3.0,yes,3.0,yes
12,541862,382364,mobile,ended,125.0,540607.0,0.0,checkout delayed,1.0,yes,1.0,yes
37,559781,408776,mobile,ended,44.0,540868.0,0.0,checkout delayed,26.0,yes,26.0,yes
64,574568,301512,mobile,ended,110.0,572909.0,0.0,checkout delayed,13.0,yes,13.0,yes
68,535519,353425,connect,ended,-166.0,533413.0,0.0,checkout not delayed,4.0,yes,4.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...
1804,561206,312603,connect,ended,10.0,554958.0,30.0,checkout delayed,183.0,yes,153.0,yes
1806,561476,410402,mobile,ended,11.0,550186.0,0.0,checkout delayed,21.0,yes,21.0,yes
1812,560740,383344,connect,ended,17.0,534678.0,60.0,checkout delayed,164.0,yes,104.0,yes
1823,566008,378689,mobile,canceled,NaN,564756.0,0.0,checkout not delayed,39.0,yes,39.0,yes


Let us visualize the distribution of delays in the cases where the next client was impacted.

In [34]:
# Create a violin trace
trace = go.Violin(y=impacted_df['delay_of_rental_start_in_minutes'], name='Delay Distribution', box_visible=True, meanline_visible=True)

layout = go.Layout(title='Distribution of Delays')

fig = go.Figure(data=[trace], layout=layout)

fig.show()

In [35]:
impacted_df = impacted_df.sort_values(by=['delay_of_rental_start_in_minutes'], ascending=False)
impacted_df[['rental_id', 'delay_of_rental_start_in_minutes']].head(10)

,rental_id,delay_of_rental_start_in_minutes
280,548388,12548.0
1623,564699,4018.0
526,541178,2976.0
1131,573931,1640.0
668,545099,1500.0
877,543665,871.0
1143,560412,836.0
152,552049,775.0
862,544753,747.0
693,549023,596.0


The majority of delays are shorter than 900 minutes, but there are several outliers. We will check how significant the part of delays longer than 900 minutes is.

In [36]:
# Filter the dataframe to only include delays greater than 900 minutes
delay_over_900 = impacted_df[impacted_df['delay_of_rental_start_in_minutes'] > 900]

# Calculating the percentage of delays greater than 900 minutes
percent_delay_over_900 = (len(delay_over_900) / len(df)) * 100

print(f"The percentage of delays greater than 900 minutes is: {percent_delay_over_900:.2f}%")

The percentage of delays greater than 900 minutes is: 0.02%


In [37]:
impact_less_900_min = impacted_df[impacted_df['delay_of_rental_start_in_minutes']<=900]
impact_less_900_min.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout,delay_of_previous_client_in_minutes,previous_client_late,delay_of_rental_start_in_minutes,client_rental_start_impacted
877,543665,397413,mobile,ended,7.0,541444.0,30.0,checkout delayed,901.0,yes,871.0,yes
1143,560412,336349,connect,canceled,NaN,568759.0,630.0,checkout not delayed,1466.0,yes,836.0,yes
152,552049,404192,mobile,ended,1392.0,551936.0,150.0,checkout delayed,925.0,yes,775.0,yes
862,544753,380478,mobile,ended,810.0,526537.0,0.0,checkout delayed,747.0,yes,747.0,yes
693,549023,365926,connect,canceled,NaN,550970.0,0.0,checkout not delayed,596.0,yes,596.0,yes


We'll create a histogram to see the distribution of impacts of delays on the next client (how long the next client had to wait before beginning the rental).

In [38]:
# Creating a histogram
trace = go.Histogram(x=impact_less_900_min['delay_of_rental_start_in_minutes'], nbinsx=90, marker=dict(color='blue'))

layout = go.Layout(title='How much were rentals delayed because of the previous client?', xaxis=dict(title='Delay of Rental Start in Minutes'), yaxis=dict(title='Count'))

fig = go.Figure(data=[trace], layout=layout)

fig.update_layout(bargap=0.15)

fig.show()

We'll use a histogram to see what were the impacts that made the next clients cancel the rental.

In [39]:
canceled_rental_start_delays = impact_less_900_min['delay_of_rental_start_in_minutes'][impact_less_900_min['state']=='canceled']
not_canceled_rental_start_delays = impact_less_900_min['delay_of_rental_start_in_minutes'][impact_less_900_min['state']=='ended']

In [40]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=canceled_rental_start_delays, nbinsx=90, marker=dict(color='#c71414'), name='canceled rentals'))
fig.add_trace(go.Histogram(x=not_canceled_rental_start_delays, nbinsx=90, marker=dict(color='#7b728e'), opacity=0.7, name='not canceled rentals'))
fig.update_layout(barmode='stack', bargap=0.15, title='How many rentals were canceled depending on the delay of rental start?', xaxis=dict(title='Delay of Rental Start in Minutes'), yaxis=dict(title='Count'))
fig.show()

We will also visualize the distribution of cancellations per type of check-in.

In [41]:
mobile_canceled_rental_start_delays = impact_less_900_min['delay_of_rental_start_in_minutes'][(impact_less_900_min['state']=='canceled') & (impact_less_900_min['checkin_type']=='mobile')]
mobile_not_canceled_rental_start_delays = impact_less_900_min['delay_of_rental_start_in_minutes'][(impact_less_900_min['state']=='ended')& (impact_less_900_min['checkin_type']=='mobile')]

In [42]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=mobile_canceled_rental_start_delays, nbinsx=90, marker=dict(color='#c71414'), name='canceled rentals'))
fig.add_trace(go.Histogram(x=mobile_not_canceled_rental_start_delays, nbinsx=90, marker=dict(color='#7b728e'), opacity=0.7, name='not canceled rentals'))

# Overlay both histograms
fig.update_layout(barmode='stack', bargap=0.15, title='How many rentals with mobile check-in were canceled depending on the delay of rental start?', xaxis=dict(title='Delay of Rental Start in Minutes'), yaxis=dict(title='Count'))
fig.update_layout(bargap=0.15)
fig.show()

In [43]:
connect_canceled_rental_start_delays = impact_less_900_min['delay_of_rental_start_in_minutes'][(impact_less_900_min['state']=='canceled') 
                                                                                               & (impact_less_900_min['checkin_type']=='connect')]
connect_not_canceled_rental_start_delays = impact_less_900_min['delay_of_rental_start_in_minutes'][(impact_less_900_min['state']=='ended') 
                                                                                                   & (impact_less_900_min['checkin_type']=='connect')]

In [44]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=connect_canceled_rental_start_delays, nbinsx=90, marker=dict(color='#c71414'), name='canceled rentals'))
fig.add_trace(go.Histogram(x=connect_not_canceled_rental_start_delays, nbinsx=90, marker=dict(color='#7b728e'), opacity=0.7, name='not canceled rentals'))

fig.update_layout(barmode='stack', bargap=0.15, title='How many rentals with connect check-in were canceled depending on the delay of rental start?', xaxis=dict(title='Delay of Rental Start in Minutes'), yaxis=dict(title='Count'))
fig.update_layout(barmode='stack', bargap=0.15)
fig.update_layout(bargap=0.15)
fig.show()

In [45]:
# Computing the percentage of cancellations for cases where clients were impacted by the delays
canceled = impacted_df[impacted_df['state'] == 'canceled']
not_canceled = impacted_df[impacted_df['state'] == 'ended']

mobile_canceled_pct = round(100 * len(canceled[canceled['checkin_type'] == 'mobile']) / len(impacted_df[impacted_df['checkin_type'] == 'mobile']), 2)
mobile_not_canceled_pct = round(100 * len(not_canceled[not_canceled['checkin_type'] == 'mobile']) / len(impacted_df[impacted_df['checkin_type'] == 'mobile']), 2)
connect_canceled_pct = round(100 * len(canceled[canceled['checkin_type'] == 'connect']) / len(impacted_df[impacted_df['checkin_type'] == 'connect']), 2)
connect_not_canceled_pct = round(100 * len(not_canceled[not_canceled['checkin_type'] == 'connect']) / len(impacted_df[impacted_df['checkin_type'] == 'connect']), 2)
total_canceled_pct = round(100 * len(canceled) / len(impacted_df), 2)
total_not_canceled_pct = round(100 * len(not_canceled) / len(impacted_df), 2)

# Creating a new dataframe with the percentages computed above
df_pct = pd.DataFrame({
    'Contract Type': ['Mobile', 'Connect', 'Overall'],
    'Canceled': [mobile_canceled_pct, connect_canceled_pct, total_canceled_pct],
    'Not Canceled': [mobile_not_canceled_pct, connect_not_canceled_pct, total_not_canceled_pct]
})

# Creating a horizontal stacked bar chart
fig = go.Figure()
fig.add_trace(go.Bar(y=df_pct['Contract Type'], x=df_pct['Canceled'], orientation='h', name='Canceled', marker=dict(color='#c71414'), text=[f"{mobile_canceled_pct}%", f"{connect_canceled_pct}%", f"{total_canceled_pct}%"],
                     textposition='auto', textfont=dict(color='white')))
fig.add_trace(go.Bar(y=df_pct['Contract Type'], x=df_pct['Not Canceled'], orientation='h', name='Not Canceled', marker=dict(color='#7b728e'), text=[f"{mobile_not_canceled_pct}%", f"{connect_not_canceled_pct}%", f"{total_not_canceled_pct}%"],
                     textposition='auto', textfont=dict(color='white')))
fig.update_layout(barmode='stack', title='How many of the clients impacted by the delay of the previous driver canceled their rentals?', yaxis={'categoryorder':'total ascending'})

fig.show()

Are client delays responsible for the majority of cancellations, or are most rentals canceled for other reasons?

In [46]:
# Counting the number of cases where the driver was late at checkout
total_delays = len(prev_df[prev_df["previous_client_late"]=="yes"])

connect_late = round(len(prev_df[(prev_df["checkin_type"]=="connect") 
                           & (prev_df["previous_client_late"]=="yes")]) / total_delays * 100, 2)

mobile_late = round(len(prev_df[(prev_df["checkin_type"]=="mobile") 
                          & (prev_df["previous_client_late"]=="yes")]) / total_delays * 100, 2)

mobile_late_impacted = round(len(prev_df[(prev_df["checkin_type"]=="mobile") 
                                   & (prev_df["previous_client_late"]=="yes") 
                                   & (prev_df["client_rental_start_impacted"]=="yes")]) / total_delays * 100, 2)

connect_late_impacted = round(len(prev_df[(prev_df["checkin_type"]=="connect") 
                                   & (prev_df["previous_client_late"]=="yes") 
                                   & (prev_df["client_rental_start_impacted"]=="yes")]) / total_delays * 100, 2)

mobile_late_impacted_canceled = round(len(prev_df[(prev_df["checkin_type"]=="mobile") 
                                   & (prev_df["previous_client_late"]=="yes") 
                                   & (prev_df["client_rental_start_impacted"]=="yes")
                                   & (prev_df["state"]=="canceled")]) / total_delays * 100)


connect_late_impacted_canceled = round(len(prev_df[(prev_df["checkin_type"]=="connect") 
                                   & (prev_df["previous_client_late"]=="yes") 
                                   & (prev_df["client_rental_start_impacted"]=="yes")
                                   & (prev_df["state"]=="canceled")]) / total_delays * 100, 2)


In [47]:
stages = ['canceled','next client impacted','previous driver late']
sr1 = [mobile_late_impacted_canceled, mobile_late_impacted, mobile_late] # values for mobile 
sr2= [connect_late_impacted_canceled, connect_late_impacted, connect_late] # values for connect
#convert sr1
def convert(lst):
    return [ -i for i in lst ]
sr3 = convert(sr2)
fig = go.Figure()
fig.add_trace(go.Bar(y=stages, x=sr1,
                base=0,
                marker_color='#8d1586',
                name='Mobile',
                orientation='h',
                text = sr1,
                textposition='inside',
                texttemplate = "%{x} %"
))
fig.add_trace(go.Bar(y=stages, x=sr2,
                base=sr3,
                marker_color='#eec186',
                name='Connect',
                orientation='h',
                text = sr2,
                textposition='inside',
                texttemplate = "%{x} %"
))
fig.update_layout(barmode='overlay', xaxis_tickangle=45, title_text="For both check-in types, how many cancellations are attributable to delays?")



Saving the enriched dataframes df and prev_df as csv files that will be used for streamlit visualisations

In [48]:
# Commented to avoid unnecessary rewriting
"""df.to_csv("getaround_df.csv", index=False)
prev_df.to_csv("getaround_deltas_df.csv", index=False)"""

'df.to_csv("getaround_df.csv", index=False)\nprev_df.to_csv("getaround_deltas_df.csv", index=False)'

In [49]:
# Commented to avoid unnecessary rewriting
"""import boto3
import os

ACCESS_KEY_ID = os.getenv("ACCESS_KEY_ID")
SECRET_ACCESS_KEY = os.getenv("SECRET_ACCESS_KEY")
session = boto3.Session(aws_access_key_id=ACCESS_KEY_ID, 
                        aws_secret_access_key=SECRET_ACCESS_KEY)
s3 = session.resource("s3")
bucket = s3.Bucket("getaround-bucket-27-12-2022") 
bucket.upload_file("getaround_df.csv", Key="getaround_df.csv")
bucket.upload_file("getaround_deltas_df.csv", Key="getaround_deltas_df.csv")"""


'import boto3\nimport os\n\nACCESS_KEY_ID = os.getenv("ACCESS_KEY_ID")\nSECRET_ACCESS_KEY = os.getenv("SECRET_ACCESS_KEY")\nsession = boto3.Session(aws_access_key_id=ACCESS_KEY_ID, \n                        aws_secret_access_key=SECRET_ACCESS_KEY)\ns3 = session.resource("s3")\nbucket = s3.Bucket("getaround-bucket-27-12-2022") \nbucket.upload_file("getaround_df.csv", Key="getaround_df.csv")\nbucket.upload_file("getaround_deltas_df.csv", Key="getaround_deltas_df.csv")'